# Manifold Learning for Speech Emotion Recognition
## Efthymios Tzinis

In [1]:
# Load the appropriate modules 
import os, sys, glob
import numpy as np
sys.path.append('../')
import config
sys.path.append(config.BASE_PATH)
from dataloader import fused_features_IEMOCAP as IEMOCAP_loader

sys.path.append(config.PATTERN_SEARCH_MDS_PATH)

In [2]:
# Find all appropriate files 
l_feats_p = '../../data/linear/IEMOCAP_linear_emobase2010'
nl_feats_l = glob.glob('../../data/utterance/*.dat')
nl_feats_p = nl_feats_l.pop()
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p, nl_feats_p])

In [3]:
# Session Folds Generator
def generate_folds(features_dic,
                   group_by = 'speaker'):
    if group_by == 'speaker':
        for te_speaker, te_data in features_dic.items():
            x_tr_list = []
            Y_tr = []
            for tr_speaker, tr_data in features_dic.items():
                if tr_speaker == te_speaker:
                    continue
                x_tr_list.append(tr_data['x'])
                Y_tr += tr_data['y']

            X_tr = np.concatenate(x_tr_list, axis=0)
            yield te_speaker, te_data['x'], te_data['y'], X_tr, Y_tr
     
    elif group_by == 'session':
        already_tested = []
        for te_speaker, te_data in features_dic.items():
            if not (te_speaker[:-1] in already_tested) :
                already_tested.append(te_speaker[:-1])
            else:
                continue
            X_val =  te_data['x']
            Y_val = te_data['y']
            x_tr_list = []
            Y_tr = []
            ses_name = te_speaker[:-1]
            for tr_speaker, tr_data in features_dic.items():
                if tr_speaker == te_speaker:
                    continue
                if tr_speaker[:-1] == ses_name:
                    val_speaker = tr_speaker
                    X_val = tr_data['x']
                    Y_val = tr_data['y']
                    continue
                x_tr_list.append(tr_data['x'])
                Y_tr += tr_data['y']

            X_tr = np.concatenate(x_tr_list, axis=0)
            X_ses = np.concatenate([te_data['x'], X_val], axis=0)
            Y_ses = te_data['y'] + Y_val
            yield ses_name, X_ses, Y_ses, X_tr, Y_tr
            
def fuse_excited_happiness(l):
    return ['happy + excited' 
            if (e == 'excited' or e == 'happy') 
            else e for e in l ]


In [4]:
# Initialize all avaiulable Manifold Methods
import multidimensional
import multidimensional.common
import multidimensional.mds 
import multidimensional.smacof
from sklearn import manifold, decomposition

def get_manifold_methods(target_dim):
    method_n_comp = 66
    radius_barrier = 1e-3
    explore_dim_percent = .9
    starting_radius = 8
    max_turns = 10000
    point_filter = (multidimensional.point_filters.FixedStochasticFilter(keep_percent=1, recalculate_each=10))
    radius_update = (multidimensional.radius_updates.AdaRadiusHalving(tolerance=.5*1e-3, burnout_tolerance=100000))

    mds_obj = multidimensional.mds.MDS(target_dim, point_filter, radius_update, starting_radius=starting_radius, 
                                       radius_barrier=radius_barrier,
                max_turns=max_turns, keep_history=False,
                explore_dim_percent=explore_dim_percent)

    manifold_methods = {
        'Pattern Search MDS': { 'results': {}, 'object': multidimensional.mds.MDS(target_dim, point_filter, 
                                                         radius_update, starting_radius=starting_radius, 
                                                         radius_barrier=radius_barrier, max_turns=max_turns, 
                                                         keep_history=False,
                                                         explore_dim_percent=explore_dim_percent)},
        'MDS SMACOF': { 'results': {}, 'object': multidimensional.smacof.MDS(n_components=target_dim, n_init=1, 
                                                 max_iter=max_turns, dissimilarity='euclidean', n_jobs=8)},
        'LTSA': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='ltsa',n_jobs=8)},
        'Modified LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='modified',n_jobs=8)},
        'Hessian LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='hessian',n_jobs=8)},
        'LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='standard',n_jobs=8)},
        'Truncated SVD': { 'results': {}, 'object': decomposition.TruncatedSVD(n_components=target_dim)},
        'Spectral Embedding': { 'results': {}, 'object': manifold.SpectralEmbedding(n_components=target_dim, 
                                                                                    n_jobs=8)},
#         'TSNE': { 'results': {}, 'object': manifold.TSNE(n_components=target_dim)},

    }
    return manifold_methods

In [5]:
# Find the best performing nonlinear features for KNN classification after dimensionality reduction
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pprint 


n_neighbors = np.arange(1,30)
target_dims = [10]

for target_dim in target_dims:
    manifold_methods = get_manifold_methods(target_dim)

    obj = manifold_methods['Truncated SVD']['object']
    
    session_folds = generate_folds(data_dic, group_by='session')
    for session, X_te, Y_te, X_tr, Y_tr in session_folds:
        print session
        Y_te, Y_tr = fuse_excited_happiness(Y_te), fuse_excited_happiness(Y_tr)
        Y_all = Y_te + Y_tr
        
        test_indices = 500
        Y_te = Y_all[:test_indices]
        Y_tr = Y_all[test_indices:1000]

        X_new = np.concatenate([X_te, X_tr], axis=0)
        # normalize the input vectors 
        X_high = StandardScaler().fit_transform(X_new[:1000])

        print 'Finding Embedding...'
        print X_high.shape
        X_low = obj.fit_transform(X_high)
        
        for k in n_neighbors:
            knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', algorithm='brute', leaf_size=30, 
                                       p=2, metric='minkowski', metric_params=None, n_jobs=8)

            knn.fit(X_low[test_indices:], Y_tr) 
            Y_predicted = knn.predict(X_low[:test_indices])

            w_acc = accuracy_score(Y_predicted, Y_te)
            cmat = confusion_matrix(Y_te, Y_predicted)
            with np.errstate(divide='ignore'):
                uw_acc = (cmat.diagonal() / (1.0 * cmat.sum(axis=1) + 1e-6)).mean()
                if np.isnan(uw_acc):
                    uw_acc = 0.

            metrics_l = [('uw_acc', uw_acc), ('w_acc', w_acc)]
            print "Neighbors = {}".format(k)
            pprint.pprint(metrics_l) 

Ses05
Finding Embedding...
(1000, 2014)
Neighbors = 1
[('uw_acc', 0.41843417003848843), ('w_acc', 0.45200000000000001)]
Neighbors = 2
[('uw_acc', 0.44703816142682357), ('w_acc', 0.498)]
Neighbors = 3
[('uw_acc', 0.45565510798885966), ('w_acc', 0.48399999999999999)]
Neighbors = 4
[('uw_acc', 0.4985850796737481), ('w_acc', 0.53800000000000003)]
Neighbors = 5
[('uw_acc', 0.46849964581481063), ('w_acc', 0.51200000000000001)]
Neighbors = 6
[('uw_acc', 0.48489460364519088), ('w_acc', 0.52800000000000002)]
Neighbors = 7
[('uw_acc', 0.49521813299739936), ('w_acc', 0.54000000000000004)]
Neighbors = 8
[('uw_acc', 0.49419047198100335), ('w_acc', 0.54600000000000004)]
Neighbors = 9
[('uw_acc', 0.49899614414105586), ('w_acc', 0.54200000000000004)]
Neighbors = 10
[('uw_acc', 0.50268066793131105), ('w_acc', 0.54600000000000004)]
Neighbors = 11
[('uw_acc', 0.52309348290959734), ('w_acc', 0.57199999999999995)]
Neighbors = 12
[('uw_acc', 0.52403465936158344), ('w_acc', 0.57199999999999995)]
Neighbors = 

Neighbors = 17
[('uw_acc', 0.42939530404884618), ('w_acc', 0.41399999999999998)]
Neighbors = 18
[('uw_acc', 0.44069322481512468), ('w_acc', 0.42799999999999999)]
Neighbors = 19
[('uw_acc', 0.43336455804958635), ('w_acc', 0.41799999999999998)]
Neighbors = 20
[('uw_acc', 0.43442348123947139), ('w_acc', 0.41999999999999998)]
Neighbors = 21
[('uw_acc', 0.44350681856008756), ('w_acc', 0.42599999999999999)]
Neighbors = 22
[('uw_acc', 0.4353301246115816), ('w_acc', 0.41999999999999998)]
Neighbors = 23
[('uw_acc', 0.43904736248729076), ('w_acc', 0.42399999999999999)]
Neighbors = 24
[('uw_acc', 0.43931928146895094), ('w_acc', 0.42599999999999999)]
Neighbors = 25
[('uw_acc', 0.44222961227980612), ('w_acc', 0.42799999999999999)]
Neighbors = 26
[('uw_acc', 0.44870080952648667), ('w_acc', 0.436)]
Neighbors = 27
[('uw_acc', 0.44873754180248504), ('w_acc', 0.434)]
Neighbors = 28
[('uw_acc', 0.44640031995157348), ('w_acc', 0.432)]
Neighbors = 29
[('uw_acc', 0.44868836794923267), ('w_acc', 0.436)]
Ses0